In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai

# Set the API key, get from gemini
genai.configure(api_key="")

# Set up the model
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    }
]

model = genai.GenerativeModel(model_name="gemini-pro",
                               generation_config=generation_config,
                               safety_settings=safety_settings)



In [ ]:
import pandas as pd
import random

# Đọc file CSV
path = "/kaggle/input/dataset-cs221/train_en.csv"
df = pd.read_csv(path)

In [ ]:
df2 = df

In [ ]:
ids = df2["id"]
used_ids = ids

In [ ]:
import time
import pandas as pd
from random import randint

data_frame = pd.DataFrame()
i = 0
for index, row in df2.iterrows():
    start = time.time()

    text = row["text"]
    if (row['binary'] == 'Hope'):
        prompt = f"""
                    Generate a English twitter text, which is as similar as this text: {text} so that it still convey a mention of hope, only text
                  """
    elif (row['binary'] == 'Not Hope'):
        prompt = f"""
                    Generate a English twitter text, which is as similar as this text: {text} so that it does not convey a mention of hope, only text
                  """

    try:
        response = model.generate_content(prompt)
        new_text = response.text
    except ValueError as e:
        if "safety_ratings" in str(e):
            print(f"Skipping row {index+1}: Response blocked due to safety ratings.")
        else:
            print(f"Skipping row {index+1}: Error generating text ({e})")
        continue

    # Add label, build dataframe and save to csv
    text_value = new_text
    binary_value = row['binary']
    multiclass_value = row['multiclass']
    value_id = None
    while value_id is None or value_id in used_ids:
        value_id = randint(1, 10000)
    data = {
        'text': [text_value],
        'binary': [binary_value],
        'multiclass': [multiclass_value],
        'id': [value_id]
    }

    data_frame = pd.concat([data_frame, pd.DataFrame(data)], ignore_index=True)
    data_frame.to_csv("generated_data_spanish.csv", index=False)

    # End
    i = i + 1
    print(i)
    print(new_text)
    print()
    print("SUCCESSFULL")
    print()

    end = time.time()
    time.sleep(end - start + 1)


In [ ]:
data_frame.to_csv("generated_train_en.csv", index=False)